# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Clustering avancé et choix final
## Bibliothèques nécessaires

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans, HDBSCAN
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.offline as pyo
import numpy as np
from collections import Counter

## Lecture des données

In [2]:
# Chargement des jeux de données
df = pd.read_parquet(
    "../data/processed/req_tourisme_responsable_final.parquet",
    engine='pyarrow'
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,stop_words,tokens_final,tokens_lemmatized,embedding_prop,embedding_prop_x,embedding_prop_y,cluster_label,top_15_words,cluster_label_cah,top_15_words_cah
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,"['il', 'une', 'des', 'avec', 'la', 'pour', 'ou']","[surveillance, lieux, amende, clef, degradatio...","['surveillance', 'lieux', 'touristique', 'amen...","[0.026475444436073303, 0.013547265902161598, 0...",60.746189,15.809195,9,"[dechet, lieux, tri, eau, poubelle, mettre, se...",4,"[camping, dechet, eau, lieux, car, poubelle, t..."
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,"['il', 'des', 'et', 'des', 'de', 'aux', 'des',...","[donner, sac, poubelle, cendrier, poch, vacanc...","['donner', 'sac', 'poubelle', 'cendrier', 'poc...","[0.034178830683231354, 0.05750420689582825, 0....",54.815662,12.159612,7,"[camping, car, parking, lieux, aire, developpe...",4,"[camping, dechet, eau, lieux, car, poubelle, t..."
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,"['il', 'et', 'en', 'les', 'des', 'du', 'et', '...","[recompenser, mettre, avant, acte, projet, eco...","['recompenser', 'mettre', 'avant', 'acte', 'pr...","[-0.011080465279519558, 0.053166456520557404, ...",-22.491800,47.501347,4,"[responsable, developper, rendre, lieux, acteu...",1,"[mettre, responsable, pouvoir, etre, creer, gr..."


## Methode specifique : Grid Search qui optimise la metrique silhouette avec la technique HDBSCAN
## Résumé de la méthode HDBSCAN
HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) est une méthode de clustering non supervisée basée sur la densité, qui peut identifier des clusters de densité variable et marquer les points qui ne font pas partie de clusters comme du bruit (outliers). HDBSCAN est une extension de DBSCAN, et contrairement à DBSCAN, il ne nécessite pas de spécifier un seul paramètre de densité (epsilon) et est capable de gérer des clusters de différentes densités de manière plus robuste.

### Fonctionnement global de HDBSCAN
Construction de l'arbre de MST (Minimum Spanning Tree) :
HDBSCAN commence par construire un arbre couvrant minimal pondéré à partir des données en utilisant une distance de type k-distance. Cet arbre capture les relations de proximité entre les points.

### Condensation de l'arbre :
HDBSCAN condense cet arbre en supprimant progressivement les bords les moins denses (ceux qui relient les points les plus éloignés), ce qui permet de détecter des clusters naturels dans les données.

### Extraction des clusters :
Les clusters sont extraits de cet arbre condensé en cherchant les regroupements de points ayant une densité suffisante.

### Attribution de labels et détection des outliers :
Les points qui ne sont pas assez proches de leurs voisins pour appartenir à un cluster sont marqués comme outliers.

### Paramètres de HDBSCAN
Voici les principaux paramètres de HDBSCAN que vous pouvez optimiser avec une recherche par grille :
- min_cluster_size :  
    C'est la taille minimale d'un cluster. Cela détermine le nombre minimum de points nécessaires pour former un cluster dense.  
    Un plus grand min_cluster_size peut rendre l'algorithme moins sensible aux petits clusters et réduire le bruit, tandis qu'un plus petit min_cluster_size peut détecter plus de petits clusters.

- min_samples :
    C'est le nombre minimum de points dans le voisinage d'un point pour qu'il soit considéré comme un noyau de cluster. Par défaut, min_samples est égal à min_cluster_size, mais il peut être ajusté indépendamment.  
    Un plus grand min_samples augmente la robustesse contre le bruit mais peut nécessiter des clusters plus denses pour être détectés.

- metric :
    La métrique de distance utilisée pour calculer les distances entre les points (par exemple, 'euclidean', 'manhattan', etc.).  
    La métrique de distance choisie peut influencer la forme et la taille des clusters détectés. Le choix de la métrique dépend de la nature des données et des relations que vous souhaitez capturer.

## Qu'est-ce que le Silhouette Score ?
Le Silhouette Score est une métrique utilisée pour évaluer la qualité des clusters dans un ensemble de données. Il mesure à quel point un point de données est bien associé à son propre cluster par rapport aux autres clusters. La silhouette d'un point de données est une combinaison de deux scores :

Cohésion (a) : La distance moyenne entre un point et tous les autres points du même cluster.
Séparation (b) : La distance moyenne entre un point et tous les points du cluster le plus proche auquel il n'appartient pas.
La silhouette pour un point de données est définie comme suit :

$$\ s = \frac{b - a}{\max(a, b)} \$$

où :

- \( a \) est la distance moyenne entre le point et les autres points du même cluster (Cohésion).
- \( b \) est la distance moyenne entre le point et les points du cluster le plus proche qui n'est pas son propre cluster (Séparation).

Le Silhouette Score global pour un ensemble de données est la moyenne des silhouettes de tous les points de données.

### Interprétation du Silhouette Score
1 : Le point est bien attribué à son propre cluster et est loin des autres clusters.  
0 : Le point se trouve à la frontière entre deux clusters.  
-1 : Le point est mal attribué à un cluster et devrait plutôt appartenir à un autre cluster.


### Utilisation pratique du Silhouette Score
Le Silhouette Score est particulièrement utile pour :
- Évaluer le nombre de clusters :
    * Comparer les Silhouette Scores pour différents nombres de clusters peut aider à déterminer le nombre optimal de clusters.
    
    * Comparer des algorithmes de clustering : Permet de comparer la qualité des clusters formés par différents algorithmes de clustering.

### Exécution de la Grid Search pour optimiser la silhouette
L'objectif de la recherche par grille (Grid Search) est de trouver la meilleure combinaison de ces paramètres en maximisant une métrique de performance, telle que le score de silhouette. Voici un exemple de code pour effectuer cette optimisation :

### Définition des plages de paramètres et initialisation

In [3]:
# Suppose que vectors est votre ensemble de vecteurs
# Paramètres à tester

min_samples_values = 6

min_cluster_size_values = range(3,10,1)
max_cluster_size_values = range(200,500,5)
metric_values = ['euclidean', 'manhattan']

# Variables pour stocker les meilleurs paramètres et le score correspondant
best_score = -1
best_params = {}

# encadrement du nombre de cluster
min_clusters_required = 5
max_clusters_required = 20

# Facteur de pondération pour la pénalisation des points de bruit
alpha = 2

### Création d'une matrice d'embeddings

In [4]:
# Convertir la colonne 'embedding_prop' en une seule matrice
embedding_matrix = np.stack(df['embedding_prop'].values)
embedding_matrix.shape

(1493, 20)

### Grid Search pour HDBSCAN

In [5]:
for min_cluster_size in min_cluster_size_values:
    for max_cluster_size in max_cluster_size_values:
        for metric in metric_values:
            # Initialiser HDBSCAN avec les paramètres actuels
            clusterer = HDBSCAN(min_cluster_size=min_cluster_size,
                            max_cluster_size=max_cluster_size,
                            min_samples = min_samples_values,
                            metric=metric)
            # Ajuster le modèle
            cluster_labels = clusterer.fit_predict(embedding_matrix)

            # Compter le nombre de points de bruit (-1)
            num_noise_points = np.sum(cluster_labels == -1)
            
            # Nombre total de point
            total_points = len(cluster_labels)

            # Compter le nombre de clusters formés (ignorer les points de bruit avec le label -1)
            unique_labels = set(cluster_labels)
            if -1 in unique_labels:
                unique_labels.remove(-1)
            num_clusters = len(unique_labels)
            
            # Vérifier si le nombre de clusters est dans les bornes spécifiées
            if min_clusters_required <= num_clusters <= max_clusters_required:
                valid_labels = cluster_labels[cluster_labels >= 0]
                valid_vectors = embedding_matrix[cluster_labels >= 0]

                # Score silhouette moyen
                silhouette_avg = silhouette_score(valid_vectors, valid_labels)
                # Penalité du au nombre de points isolés
                noise_penalty = alpha * (num_noise_points / total_points)
                # metric àf" optimiser
                composite_score = silhouette_avg - noise_penalty

                print(f"Params: min_cluster_size={min_cluster_size}, metric={metric},"
                    f"Composite Score={composite_score}, Silhouette Score={silhouette_avg},"
                    f"Number of Clusters={num_clusters}, Number of Noise Points={num_noise_points}")
                # Vérifier si c'est le meilleur score
                if composite_score > best_score:
                    best_score = composite_score
                    best_params = {
                        'min_cluster_size': min_cluster_size,
                        'max_cluster_size': max_cluster_size,
                        'metric': metric
                    }

print("Meilleurs paramètres : ", best_params)
print("Meilleur score de silhouette : ", best_score)

Params: min_cluster_size=3, metric=euclidean,Composite Score=-1.612945187439066, Silhouette Score=0.12985454464398835,Number of Clusters=5, Number of Noise Points=1301
Params: min_cluster_size=3, metric=manhattan,Composite Score=-1.507918461492, Silhouette Score=0.21076874547384047,Number of Clusters=5, Number of Noise Points=1283
Params: min_cluster_size=3, metric=euclidean,Composite Score=-1.612945187439066, Silhouette Score=0.12985454464398835,Number of Clusters=5, Number of Noise Points=1301
Params: min_cluster_size=3, metric=manhattan,Composite Score=-1.507918461492, Silhouette Score=0.21076874547384047,Number of Clusters=5, Number of Noise Points=1283
Params: min_cluster_size=3, metric=euclidean,Composite Score=-1.612945187439066, Silhouette Score=0.12985454464398835,Number of Clusters=5, Number of Noise Points=1301
Params: min_cluster_size=3, metric=manhattan,Composite Score=-1.507918461492, Silhouette Score=0.21076874547384047,Number of Clusters=5, Number of Noise Points=1283
P

## Test et affichage de cette technique de clustering

In [6]:
# Initialiser HDBSCAN avec les paramètres actuels
best_hdbscan = HDBSCAN( min_cluster_size=3,
                        max_cluster_size = 200,
                        min_samples=6)
# Ajuster le modèle
cluster_labels = best_hdbscan.fit_predict(embedding_matrix)

# Ajouter les étiquettes des clusters à votre dataframe
df['cluster_label_hdbscan'] = cluster_labels

In [7]:
df['cluster_label_hdbscan'].value_counts()

cluster_label_hdbscan
-1    1301
 3     175
 0       6
 2       5
 4       3
 1       3
Name: count, dtype: int64

In [8]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(
    df[['content', 'embedding_prop_x', 'embedding_prop_y', 'cluster_label_hdbscan']], 
    x='embedding_prop_x', 
    y='embedding_prop_y', 
    hover_name='content',
    color='cluster_label_hdbscan', 
    color_continuous_scale='Viridis',  # Choisir une palette de couleurs
    range_color=[0, df['cluster_label_hdbscan'].max()],  # Définir la plage de couleurs en fonction des étiquettes de cluster
    labels={'cluster_label_hdbscan': 'Cluster Label'},  # Renommer l'étiquette de la légende
    color_continuous_midpoint=int(df['cluster_label_hdbscan'].max() / 2),  # Point médian de la palette de couleurs
    opacity=0.7
)
fig.update_traces(textposition='top center', showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text='2D Sentence Embeddings and Clusters (HDBSCAN)',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    coloraxis_showscale=False,
)

fig.show()


Cette technique n'est pas intéressante ici.  
Avec HDBSCAN, pour des vecteurs de mots, je pense que c'est très compliqué à mettre en œuvre et ça n'a pas grand intérêt.

### Grid search de K-Means

In [9]:
# Paramètres à tester dans la recherche par grille
n_clusters_values = range(3,20,1)  # Nombre de clusters à tester
random_state = 42  # Seed aléatoire pour l'initialisation

# Variables pour stocker les meilleurs paramètres et le score correspondant
best_score = -1
best_params = {}

In [10]:
for n_clusters in n_clusters_values:
    # Initialiser KMeans avec les paramètres actuels
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    # Ajuster le modèle
    cluster_labels = kmeans.fit_predict(embedding_matrix)

    # Calculer le score de silhouette
    silhouette_avg = silhouette_score(embedding_matrix, cluster_labels)

    # Afficher les résultats
    print(f"Params: n_clusters={n_clusters}, random_state={random_state}, Silhouette Score={silhouette_avg}")

    # Vérifier si c'est le meilleur score
    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_params = {
            'n_clusters': n_clusters,
            'random_state': random_state
        }

print("Meilleurs paramètres : ", best_params)
print("Meilleur score de silhouette : ", best_score)


Exception in thread Thread-5 (_readerthread):
Traceback (most recent call last):
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Dev\envs\tourisme-vert-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Dev\envs\tourisme-vert-env\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x82 in position 101: invalid start byte


Params: n_clusters=3, random_state=42, Silhouette Score=0.1291002563860088
Params: n_clusters=4, random_state=42, Silhouette Score=0.11530831968338952
Params: n_clusters=5, random_state=42, Silhouette Score=0.12390127123446669
Params: n_clusters=6, random_state=42, Silhouette Score=0.11190509935198287
Params: n_clusters=7, random_state=42, Silhouette Score=0.11098489410424565
Params: n_clusters=8, random_state=42, Silhouette Score=0.10853688237294665
Params: n_clusters=9, random_state=42, Silhouette Score=0.10857389541620528
Params: n_clusters=10, random_state=42, Silhouette Score=0.09877496608915598
Params: n_clusters=11, random_state=42, Silhouette Score=0.09705782432740194
Params: n_clusters=12, random_state=42, Silhouette Score=0.0873283743012999
Params: n_clusters=13, random_state=42, Silhouette Score=0.09249348224499455
Params: n_clusters=14, random_state=42, Silhouette Score=0.08388493262468476
Params: n_clusters=15, random_state=42, Silhouette Score=0.08442760153751365
Params: 

Top 5 des meilleurs silhouette :
3 clusters
5 clusters
4 clusters
6 clusters

Prendre 5 clusters serait finalement une bonne idée s'il l'on se base sur le critère du coude.

In [11]:
kmeans = KMeans(n_clusters=5, random_state=42)

kmeans.fit(embedding_matrix)

# Obtenir les labels de cluster pour chaque phrase
labels = kmeans.labels_

# Évaluer la qualité des clusters avec la métrique silhouette
silhouette_avg = silhouette_score(embedding_matrix, labels)
print("Silhouette Score:", silhouette_avg)

# Assigner les labels de cluster à chaque ligne de votre dataframe
df['cluster_label_best'] = labels

Silhouette Score: 0.12390127123446669


In [12]:
df['cluster_label_best'].value_counts()

cluster_label_best
4    568
3    303
1    299
0    243
2     80
Name: count, dtype: int64

In [13]:
def get_top_words_by_cluster(df:pd.DataFrame, cluster_col:str, tokens_col:str, n:int = 15) -> dict:
    # Dictionnaire pour stocker les mots les plus fréquents par cluster
    top_words_by_cluster = {}

    # Grouper les données par cluster
    grouped = df.groupby(cluster_col)

    for cluster_label, group in grouped:
        # Combiner tous les tokens du cluster
        all_tokens = [token for sublist in group[tokens_col] for token in sublist]

        # Compter les occurrences de chaque token
        token_counts = Counter(all_tokens)

        # Obtenir les n mots les plus fréquents
        top_words = token_counts.most_common(n)

        # Stocker les résultats
        top_words_by_cluster[cluster_label] = [word for word, _ in top_words]

    return top_words_by_cluster

In [14]:
# Obtenir les 15 mots les plus fréquents par cluster
top_words_by_cluster = get_top_words_by_cluster(df,"cluster_label_best", "tokens_final", n=15)

In [15]:
# Ajouter une nouvelle colonne au DataFrame d'origine
df['best_top_15_words'] = df['cluster_label_best'].map(top_words_by_cluster)

In [16]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(
    df[['content', 'embedding_prop_x', 'embedding_prop_y', 'cluster_label_best', 'best_top_15_words']], 
    x='embedding_prop_x', 
    y='embedding_prop_y', 
    hover_name='content',
    hover_data={'best_top_15_words': True},
    color='cluster_label_best', 
    color_continuous_scale='Viridis',  # Choisir une palette de couleurs
    range_color=[0, df['cluster_label_best'].max()],  # Définir la plage de couleurs en fonction des étiquettes de cluster
    labels={'cluster_label_best': 'Cluster Label'},  # Renommer l'étiquette de la légende
    color_continuous_midpoint=int(df['cluster_label_best'].max() / 2),  # Point médian de la palette de couleurs
    opacity=0.7
)
fig.update_traces(textposition='top center', showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text='2D Sentence Embeddings and Clusters (K-Means optimal)',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    coloraxis_showscale=False,
)

fig.show()


In [17]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/2D Sentence Embeddings and Clusters (K-Means optimal).html', auto_open=False)

'../outputs/2D Sentence Embeddings and Clusters (K-Means optimal).html'

## Sauvegarde du dataframe avec les clusters

In [18]:
# Convertir tous les vecteurs de la colonne 'embedding_prop' en float64
df['embedding_prop'] = df['embedding_prop'].apply(
    lambda x: np.array(x, dtype=np.float64)
)

In [19]:
# Sauvegarde des jeux de données
df.to_parquet(
    "../data/processed/req_tourisme_responsable_final.parquet",
    engine='pyarrow',
    index=False
)